# Image processing
This notebook details how the video stream is processed before entering the neural network.

In [ ]:
import numpy as np
import dlib
import heartsight as hs
import cv2

In [12]:
facebox_width = 256
empty_facebox = np.zeros((facebox_width, facebox_width, 3),dtype='uint8')
face_detector = dlib.get_frontal_face_detector()

In [13]:
face_buffer_list = []

with hs.open_webcam() as webcam:

    exit_requested = False
    previous_face_rectangle = None
    while not exit_requested:
        rgb_frame = hs.capture_frame(webcam)
        gray_frame = hs.rgb_to_gray(rgb_frame)
        detected_faces = face_detector(gray_frame, 1)

        face_rectangle = detected_faces[0] if detected_faces else previous_face_rectangle

        if face_rectangle:
            x,y,w,h = hs.dlib_rectangle_to_xywh(face_rectangle)
            face_frame = cv2.resize(rgb_frame[y:y+h, x:x+w], (facebox_width, facebox_width))
            hs.refresh_display(face_frame)
            face_buffer_list.append(face_frame)
            previous_face_rectangle = face_rectangle
        
        exit_requested = hs.detect_q_key_pressed()

face_buffer = np.stack(face_buffer_list)
